# MACHINE LEARNING FOR RESEARCHERS

# Notebook 2. Advanced models for supervised learning



This notebook continues the review of supervised machine learning methods for *regression* and *classification*. 

The following contents are covered: 

<ol>
    <li> Decision Trees </li>
    <li> Ensemble methods: Random Forest </li>
    <li> Kernel methods applied to linear regression </li>
    <li> Support Vector Machines </li>
</ol>

It is highly recommended that this notebook is read and run after a first reading of the theory and in parallel with the slides available in AV. 
Note also that it is not required to develop any code. All examples are totally implemented, and therefore these notebooks have to be regarded as demonstrative material. The goal is understanding the operation of the algorithms. The notebook contains several questions that have be to submitted through AV. 

As with Notebook 1, the codes used for loading and plotting the Iris and MNIST data sets have been taken from <a href=https://github.com/ageron/handson-ml2>Geron (Github site)</a>, as well as the functions to plot the classification boundaries. Please, consult the textbook for reference. 

## Decision Trees

Decision tree (DT) are **non-linear algorithms for classification and regression**. They are considered white-box algorithms since it is easy for a human to understand why a given decision has been made, in contrast to black-box methods like neural networks. 

In DT, an **algorithm selects at each step which feature** (of those not considered before) **classifies better the training examples**. Then, for each possible value of that feature, a new node is created in the tree, and the process continues until no more features left.

A common way to select the best feature at each step is to use the
**information gain** concept.  Given a random variable which takes $K$
possible outputs $\{o_k\}$, for $k$=$1,\ldots,K$, its **entropy** is given
by:

\begin{equation}
H = -\sum_{k=1}^K p_k \log_2 p_k
\end{equation}

Given the labeled data set $\pmb{X}$, $\pmb{t}$ its entropy $H(\pmb{X},
\pmb{t})$ can be estimated as $\widehat{p_k}$ = $N_k/N$ (that is, the ratio of each **label** in the training set).

Besides, the $m^{\text{th}}$ feature takes values on a set of discrete values
$\pmb{v}_m$ = $\{v_1,\ldots,v_K\}$, where $K$ depends on the specific feature $m$. Let $\pmb{X},\pmb{t}|_{(\pmb{x})_m=v}$ denote the data set comprising only the instances where the $m^{\text{th}}$ feature takes value $v$. The **information gain** associated to this feature is:

\begin{equation}
I|_{(\pmb{x})_m} = H(\pmb{X}, \pmb{t}) - \sum\limits_{\forall v \in \pmb{v}_m} p((\pmb{x})_m=v) H(\pmb{X}, \pmb{t}|_{(\pmb{x})_m=v})
\end{equation}

where $p((\pmb{x})_m=v)$ is estimated as $\#(\pmb{X},\pmb{t}|_{(\pmb{x})_m=v})/\#(\pmb{X},\pmb{t})$, where $\#S$ denotes the number of instances in set $S$.

This DT algorithm selects at each step **the feature that maximizes the
information gain**. In other words, the feature whose knowledge reduces the most
uncertainty about the final output.

To exemplify this operation we will load an example data set and see how the algorithm behaves step by step. Later, we present the sklearn classes to easily build DTs.

### The Teaching Assistant Evaluation (TAE) data set

This data set consists of evaluations of teaching performance over three
   regular semesters and two summer semesters of 151 teaching assistant
   (TA) assignments at the Statistics Department of the University of
   Wisconsin-Madison. The scores were divided into 3 roughly equal-sized
   categories ("low", "medium", and "high") to form the class variable.

Number of Instances: 151
Number of Attributes: 6 

Attribute Information:

   0. Whether of not the TA is a native English speaker (binary)
      1=English speaker, 2=non-English speaker
   1. Course instructor (categorical, 25 categories)
   2. Course (categorical, 26 categories)
   3. Summer or regular semester (binary) 1=Summer, 2=Regular
   4. Class size (numerical)
   5. (**LABEL**): Class attribute (categorical) 1=Low, 2=Medium, 3=High


In [ ]:
import numpy as np
import requests

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/tae/tae.data'
myfile = requests.get(url)
open('datasets/tae.data', 'wb').write(myfile.content)
data = np.genfromtxt('datasets/tae.data', delimiter=',')
X_TAE,t_TAE = data[:,0:-1], data[:,-1]
X,t = X_TAE,t_TAE

In [ ]:
# For data plotting
import os

# Required to use matplotlib inside the notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
def entropy(t):
    H = 0
    values = np.unique(t)
    for value in values.tolist():
        p = np.sum(t==value)/t.shape[0]
        H -= p*np.log2(p) if p>0 else 0
    return H
        
def selectfeat(X,t):
    # Search the feature (attribute) which maximizes the informacion gain
    infgain=np.zeros([X.shape[1],])
    
    for idx in range(X.shape[1]):
        values = np.unique(X[:,idx])
        infgain[idx] = entropy(t)
        for value in values.tolist():
            infgain[idx] -= np.sum(X[:,idx]==value)/X.shape[0]*entropy(t[X[:,idx]==value])
    return np.argmax(infgain), infgain

def DT(X,t,level=0):
    attr, infgain = selectfeat(X,t)
    if infgain[attr]==0: 
        return f'is LEAF-> PREDICT {np.argmax(np.bincount(t.astype(int)))}'
        
    tree = f'\n'+f'\t'*level
    tree += f'Attribute {attr} selected. Its information gain is {infgain[attr]:.03f}\n'
    values = np.unique(X[:,attr])
    for value in values.tolist():
        tree += f'\t'*(level+1)+f'Node created for value {value} in attributte {attr} '
        tree += DT(X[X[:,attr]==value], t[X[:,attr]==value], level+1)+'\n'
    return tree
        

print(DT(X,t))
            

The most relevant attribute seems the class size (4th attribute). However, we can see some **overfitting since this attribute is not categorical, but quantitative**. Indeed, we can select some categorical levels for it. For example, from 0-10 students (small), from 11-30 (mid), and from 31-66 (big), and redo the tree:  

In [ ]:
# Data is duplicated so this cell can be executed several times with consistent results
Xaux = X.copy()
Xaux[X[:,4]<11,4]=0
Xaux[(11<=X[:,4]) & (X[:,4]<31),4]=1
Xaux[(31<=X[:,4]) & (X[:,4]<11),4]=2

print(DT(Xaux,t))

Now, the dominant attribute is the particular course to be teach. 

### The Playtennis data set

We will check how the DT is build for the Playtennis example shown in the Unit 1 slides. The following codification will be used:

Attributes: 
0. Outlook: Sunny(0), Overcast(1), Rain(2)
1. Temperature: Hot(0), Mild(1), Cool(2)
2. Humidity: High(0), Normal(1)
3. Wind: Weak(0), Strong(1)
4. (**LABEL**) Playtennis: No(0), Yes(1)

In [ ]:
# Let's hard-code the data set

X = np.array([[0,0,0,0],
    [0,0,0,1],
    [1,0,0,0],
    [2,1,0,0],
    [2,2,1,0],
    [2,2,1,1],
    [1,2,1,1],
    [0,1,0,0],
    [0,2,1,0],
    [2,1,1,0],
    [0,1,1,1],
    [1,1,0,1],
    [1,0,1,0],
    [2,1,0,1]])

t = np.array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])
print(DT(X,t))

 It can be seen that the grown tree matches the one shown in the slides' example. Besides the information gain for the outlook attribute matches the one in the computations performed also in the slides. 

***
### Question 1
> **Observing the resulting DT, provide the class prediction for "Rain, Mild, High, Weak" instance, and indicate which nodes of the tree are traversed (i.e., which attributes are taken into account to make the decision).**
***

### DT with sklearn

Next, we will examine how to implement DT directly with sklearn with the class **DecisionTreeClassifier**. In this case results are different because, even using entropy as the criterion to build the tree, the algortithm used by sklearn is CART, and the one presented in the theory is ID3. Anyway, the tree formation and the general ideas remain. You can consult [this link](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) for a short discussion about these algorithms.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# To use entropy select criterion='entropy', by default it uses gini impurity
clf = DecisionTreeClassifier(criterion='gini',random_state=0).fit(X,t)
print(f'Prediction for rain, mild day with high pressure and weak wind is {"No" if (clf.predict(np.array([2,1,0,0]).reshape(1,-1))==0) else "Yes"}')

### TAE data set with sklearn 

In [ ]:
import numpy as np
import requests

X,t = X_TAE,t_TAE

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# To use entropy select criterion='entropy', by default it uses gini impurity
clf = DecisionTreeClassifier(criterion='gini',random_state=0)
print(cross_val_score(clf, X, t, cv=5)) # Does CV runs with separated validataion and training sets. 
                                 # Output is the accuracy over the validation set

clf_entropy = DecisionTreeClassifier(criterion='entropy',random_state=0)
print(cross_val_score(clf_entropy, X, t, cv=5)) # Does CV runs with separated validataion and training sets. 
                                 # Output is the accuracy over the validation set

### Sperical vs. Torus data set

In [ ]:
## Function to plot decision boundaries

from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

In [ ]:
# Create data set
#
np.random.seed(1)
r = np.random.rand(50, 1) #0...1
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_0 = np.c_[r*np.cos(a), r*np.sin(a)]

r = 0.9 + np.random.rand(50, 1) #0.9...1.9
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_1 = np.c_[r*np.cos(a), r*np.sin(a)]

X = np.r_[X_0, X_1]
t = np.r_[np.zeros([50,1]),np.ones([50,1])].ravel()

# Create DT 
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0).fit(X,t)

# Show resulting boundaries
plt.figure(figsize=(8, 4))
plot_decision_boundary(clf, X, t, axes=[-2,2,-2,2], iris=False)

### The Wedge data set

In [ ]:
np.random.seed(2)
X0 = 10*np.random.rand(100, 1) 
X1 = 10*np.random.rand(100, 1) 
X = np.c_[X0,X1]

t = np.logical_and(X[:,0]>X[:,1]**(1/1.5),X[:,0]<X[:,1]**(1.5)).ravel()
clf = DecisionTreeClassifier(random_state=0).fit(X,t)

# Show resulting boundaries
plt.figure(figsize=(8, 4))
plot_decision_boundary(clf, X, t, axes=[0,10,0,10], iris=False)

As can be seen the boundaries are formed by a combination of squared shapes.

## Ensemble methods: Random Forest

An **ensemble** is a learning structure which **combines many classifiers and makes predictions as the majority vote** of all classifiers. A **random forest is an ensemble formed by decision trees** trained with the different training data (selected at random from the total training data). This method is **non-linear** and they can be used both for regression and classification. 

First, we will show the results for the two previous examples and next for the California House data set already used in the Notebook 1. 

### Shperical vs. Torus data set

In [ ]:
np.random.seed(1)
r = np.random.rand(50, 1) #0...1
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_0 = np.c_[r*np.cos(a), r*np.sin(a)]

r = 0.9 + np.random.rand(50, 1) #0.9...1.9
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_1 = np.c_[r*np.cos(a), r*np.sin(a)]

X = np.r_[X_0, X_1]
t = np.r_[np.zeros([50,1]),np.ones([50,1])].ravel()

# Create Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=0).fit(X,t)

# Show resulting boundaries
plt.figure(figsize=(8, 4))
plot_decision_boundary(clf, X, t, axes=[-2,2,-2,2], iris=False)

### The Wedge data set

In [ ]:
np.random.seed(2)
X0 = 10*np.random.rand(100, 1) 
X1 = 10*np.random.rand(100, 1) 
X = np.c_[X0,X1]

t = np.logical_and(X[:,0]>X[:,1]**(1/1.5),X[:,0]<X[:,1]**(1.5)).ravel()
clf = RandomForestClassifier(random_state=0).fit(X,t)

# Show resulting boundaries
plt.figure(figsize=(8, 4))
plot_decision_boundary(clf, X, t, axes=[0,10,0,10], iris=False)

***
### Question 2
> **Explain why the Random Forest may have non-squared decision boundaries.**
***

### The California House data set

Decision trees and Random forest can be also used in **regression contexts**. We will exemplify their use for the example of the California House data set.

In [ ]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()

As in Notebook 1, the raw table is prepared for its use in the regression model. For that the following steps are done:

<ol>
    <li> Computation of representative features (e.g. rooms_per_household)
    <li> Separation of targets and features
    <li> Substitution of missing features (e.g. number of rooms) for the median value of the column
    <li> Substitution of categorical variables (e.g. ocean_proximity) for a 1-K encoding
    <li> Separation of training and test sets
</ol>

You can skip most of this step since it involves dataframes and ndarrays manipulation. The significant part is that at the end result is divided in training and testing sets, and targets are provided in separated vectors as well.  

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

housing = load_housing_data() # Since we perform destructive change, we reload the data
targets = housing["median_house_value"].copy()
housing = housing.drop("median_house_value", axis=1) # drop labels for training set

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


housing_num = housing.drop("ocean_proximity", axis=1)

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)
X_train_CALHOUSE, X_test_CALHOUSE, t_train_CALHOUSE, t_test_CALHOUSE = train_test_split(housing_prepared, targets, test_size=0.2) # Reserve 20% of instances for testing purposes
X_train, X_test, t_train, t_test = X_train_CALHOUSE, X_test_CALHOUSE, t_train_CALHOUSE, t_test_CALHOUSE

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

reg = RandomForestRegressor(random_state=0).fit(X_train,t_train)

y_train = reg.predict(X_train)
y_test = reg.predict(X_test)
rmse_train = np.sqrt(mean_squared_error(t_train, y_train))
rmse_test = np.sqrt(mean_squared_error(t_test, y_test))

print(f'The score (rmse) in the training set is {rmse_train:.00f}, with accuracy of {reg.score(X_train,t_train)*100:.02f}%')
print(f'The score (rmse) in the test set is {rmse_test:.00f}, with accuracy of {reg.score(X_test,t_test)*100:.02f}%')

Results is **much better** that the one achieved with a regularized linear regression!!

## Kernel Linear Regression

In Notebook 1 we have seen that the linear regression model assumes a parametric hypothesis of the form:

  \begin{equation}
  y(\pmb{x}, \pmb{w}) = \sum_{j=0}^{M-1} w_j \phi_j(\pmb{x}) = \pmb{w}^T \pmb{\phi}(\pmb{x})
  \end{equation}

The parameters $\pmb{w}$ can be determined by minimizing the **regularized squared error**. For the particular case of Ridge regression ($q$=2) we got the loss:

\begin{equation}
 \widetilde{J}(\pmb{w})  = \frac{1}{2} \sum_{n=1}^N (\pmb{w}^T \pmb{\phi}(\pmb{x}_n)- t_n)^2 + \frac{\lambda}{2} \pmb{w}^T\pmb{w}
  \end{equation}

Leading to the normal equations: 

\begin{equation}
  \pmb{w}^* = (\lambda I + \pmb{\Phi}^T \pmb{\Phi})^{-1} \pmb{\Phi}^T \pmb{t}
\end{equation}

which can be recast as:

\begin{equation*}
  \pmb{w}^* = \pmb{\Phi}^T \pmb{a}
  \end{equation*}

  where $\pmb{a}$ is the vector defined by the set $\{a_n = -\frac{1}{\lambda} (\pmb{w}^T\pmb{\phi}(\pmb{x}_n) - t_n)\}$. 
  
By using $\pmb{a}$ as the optimization problem variable (instead of $\pmb{w}$) we got the dual representation form for the loss:

  \begin{equation}
  \tilde{J}(\pmb{a}) = \frac{1}{2} \pmb{a}^T \pmb{K}\pmb{K}\pmb{a} - \pmb{a}^T\pmb{K}\pmb{t}+\frac{1}{2} \pmb{t}^T \pmb{t} + \frac{\lambda}{2}
  \pmb{a}^T\pmb{K}\pmb{a}
  \end{equation}

where $\pmb{K}$ is the [**Gram matrix**](https://en.wikipedia.org/wiki/Gramian_matrix): $K$ = $\pmb{\Phi} \pmb{\Phi}^T$. That is, $(\pmb{K})_{nm}$ = $\pmb{\phi}(\pmb{x}_n)^T \pmb{\phi}(\pmb{x}_m)$ = $k(\pmb{x}_n,\pmb{x}_m)$.

And the predictions are computed in the dual form as:

 \begin{equation}
  y(\pmb{x},\pmb{w}) = \pmb{w}^T\pmb{\phi}(\pmb{x}) = \pmb{a}^T\pmb{\Phi}\pmb{\phi}(\pmb{x}) =
  \pmb{k}(\pmb{x})^T(\pmb{K}+\lambda\pmb{I})^{-1}\pmb{t}
  \label{DUALPRED}
  \end{equation}

  Note that previous equation doesn't require the use of parameters the $\pmb{w}$, but
instead we have recast our **parametric-based model** problem to an **instance-based** method (prediction is provided in terms of the *kernel evaluations of point $\pmb{x}$ against each training point*). In addition, data conversion to/from the feature space can be avoided using the **kernel
  trick**, as will be illustrated below.
  
First, the data set used in the Notebook 1 will be created:



In [ ]:
# Libraries required
import numpy as np
import os

# Required to use matplotlib inside the notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# We set a fixed seed to get always the same results
np.random.seed(1)

# Data set generation
X = np.linspace(0,1,num=10)
X = np.array([X]).T # Since X is single-dimensional, we have to convert it to a matrix to transpose it 
t = np.sin(2*np.pi*X) + 0.1*np.random.randn(10, 1)
xfull = np.linspace(0,1,num=100);
xfull = np.array([xfull]).T
treal = np.sin(2*np.pi*xfull);

# Data set plotting
plt.plot(X, t, "b.")
plt.plot(xfull, treal, "g-")
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$t$", rotation=0, fontsize=18)
plt.axis([0, 1, -1.5, 1.5])
plt.show()

### Linear kernel

The simplest approximation is working with the *linear kernel*: $k(\pmb{x},\pmb{x}')$ = $\pmb{x}^T \pmb{x}'$ 

In [ ]:
def predict(X, t, x, L, kernel): # Now prediction is done with data set X,t as argument. No "fit" necessary
                                 # kernel can be any function created latter
    K = kernel(X,X)
    y = kernel(X,x).T.dot(np.linalg.inv(K+L*np.eye(K.shape[0]))).dot(t)
    return y

Let's check how it works in the toy data set: 

In [ ]:
def linK(X,XX): # Both arguments are matrices (one instance per row)
    return X.dot(XX.T)

y = predict(X, t, xfull, L=np.exp(-10), kernel=linK)

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

### Polynomial kernel with 2-Degree terms

Now, let's consider  $k(\pmb{x},\pmb{x}')$ = $(\pmb{x}^T \pmb{x}')^2$.

As shown in the slides, this is equivalent to use the feature space $\pmb{\phi}(\pmb{x})$=$(x_1^2,\sqrt{2}x_1 x_2,
  x_2^2)^T$, but **without explicit conversions to or from it**.

In [ ]:
def pol2K(X,XX): # Both arguments are matrices (one instance per row)
    return (X.dot(XX.T))**2

y = predict(X, t, xfull, L=np.exp(-8), kernel=pol2K)

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

A bit better! Let's try with some $\infty$-dimensional kernels (yes, this is possible and easy with the kernel trick!).

### RBF kernel

The RBF kernel is defined by:

\begin{equation}
k(\pmb{x},\pmb{x}') = \exp{\big(\frac{-\|\pmb{x}-\pmb{x}'\|^2}{2\sigma^2}\big)}
\end{equation}

The associated feature space has infinite dimensionality. Let's implement it. For that, note that: $\|\pmb{x}-\pmb{x}'\|^2$ = $\pmb{x}^T\pmb{x} + (\pmb{x}')^T\pmb{x}' - 2\pmb{x}^T\pmb{x}'$

In [ ]:
SIGMA = np.exp(0)

def gaussK(X,XX): # Both arguments are matrices (one instance per row)
    global SIGMA
    aux = np.add(X.dot(X.T).diagonal().T.reshape([-1,1]),XX.dot(XX.T).diagonal().reshape([1,-1]))
    return np.exp(-(aux-2*X.dot(XX.T))/(2*SIGMA))

y = predict(X, t, xfull, L=np.exp(-8), kernel=gaussK)

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

### Kernels with sklearn

Next, we will discuss how to use kernels directly with the sklearn library. The class which provides it is called **KernelRidge** and its use its quite straighforward:

In [ ]:
from sklearn.kernel_ridge import KernelRidge

ridge = KernelRidge(alpha=np.exp(-10), kernel='linear')
ridge.fit(X, t) # Just precomputes kernel(X,X) which will be needed in predict
y = ridge.predict(xfull) 

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

Let's try other kind of kernels (check [here the list of available kernels](https://scikit-learn.org/stable/modules/metrics.html)):

In [ ]:
from sklearn.kernel_ridge import KernelRidge

ridge1 = KernelRidge(alpha=np.exp(-10), kernel='rbf') # k(x,x') = exp(-gamma*||x-x'||^2)
ridge2 = KernelRidge(alpha=np.exp(-10), kernel='poly', degree=20) 
ridge3 = KernelRidge(alpha=np.exp(-10), kernel='poly', degree=5) 
ridge4 = KernelRidge(alpha=np.exp(-10), kernel='sigmoid') # k(x,x') = tanh(gamma*x^T x' + c_0)
ridge1.fit(X, t) # Just precomputes kernel(X,X) which will be needed in predict
ridge2.fit(X, t)
ridge3.fit(X, t) 
ridge4.fit(X, t)
y1 = ridge1.predict(xfull)
y2 = ridge2.predict(xfull)
y3 = ridge3.predict(xfull)
y4 = ridge3.predict(xfull)

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12,8))
fig.tight_layout()

ax[0,0].set_title(f'RBF kernel')
ax[0,0].plot(xfull, y1, "r-")
ax[0,1].set_title(f'Poly kernel, degree=20')
ax[0,1].plot(xfull, y2, "m-")
ax[1,0].set_title(f'Poly kernel, degree=5')
ax[1,0].plot(xfull, y3, "k-")
ax[1,1].set_title(f'Sigmoidal kernel')
ax[1,1].plot(xfull, y4, "y-")

ax[0,0].plot(xfull, treal, "g-")
ax[0,1].plot(xfull, treal, "g-")
ax[1,0].plot(xfull, treal, "g-")
ax[1,1].plot(xfull, treal, "g-")

ax[0,0].plot(X, t, "b.")
ax[0,1].plot(X, t, "b.")
ax[1,0].plot(X, t, "b.")
ax[1,1].plot(X, t, "b.")

ax[0,0].axis([0, 1, -1.5, 1.5])
ax[0,1].axis([0, 1, -1.5, 1.5])
ax[1,0].axis([0, 1, -1.5, 1.5])
ax[1,1].axis([0, 1, -1.5, 1.5])

plt.show()

Different kernels have different hyper-parameters. In order to find the best ones and the regularization coefficient ($\lambda$) we can use cross-validation with sklearn too. In this case there is no the equivalent to **LinearRegressionCV** in the library so it's necessary to implement it. The **GridSearchCV** can be used for this purpose (and in any other cross-validation search):  

In [ ]:
from sklearn.model_selection import GridSearchCV

model = GridSearchCV(KernelRidge(kernel='poly'), 
             cv=5, 
             param_grid={"alpha": np.exp(np.arange(-8,-6,0.1)), "degree": np.arange(1, 20, 1)})
model.fit(X, t)
y = model.predict(xfull)
print(f'The best model is: {model.best_params_}')

plt.plot(xfull, y, "r-")
plt.plot(xfull, treal, "g-")
plt.plot(X, t, "b.")
plt.axis([0, 1, -1.5, 1.5])
plt.show()

***
### Question 3
> **Have a look at the predict function implemented above. Is there any way to speed up its operation by  precomputing some of the computations? In other words, which operations do not depend on the new data instance and can be moved outside predict?**
***

## Support Vector Machines

SVMs are **linear** classification models of **maximum-margin** for which the dual form (instance-based) is **sparse**. In the slides we have seen that the primal form to find the boundary weights $\pmb{w}$ is:

\begin{equation}
\min\limits_{\pmb{w},b} \frac{1}{2} \|\pmb{w}\|^2
\label{PRIMAL}
\end{equation}

subject to $t_n [\pmb{w}^T \pmb{\phi}(\pmb{x}_n) + b] \ge 1$. The prediction for a new point $\pmb{x}$ corresponds to $y(\pmb{x}) = \text{sign}(\pmb{w}^T \pmb{\phi}(\pmb{x}) + b)$.

Whereas, the dual form is given by:

\begin{equation}
\max\limits_{\pmb{a}} \widetilde{L}(\pmb{a}) = \sum_{n=1}^N a_n - \frac{1}{2} \sum_{n=1}^N \sum_{m=1}^N a_na_mt_nt_mk(\pmb{x}_n,  \pmb{x}_m)
\label{DUAL}
\end{equation}

subject to $a_n\ge0$, for $n=1,\ldots,N$, and $\sum_{n=1}^N a_nt_n = 0$.


Both the primal and dual forms are [quadratic programming problems](https://en.wikipedia.org/wiki/Quadratic_programming), thus convex, and since the region
determined by the constraints is convex, there is a single
optimal.

Solving quadratic problems has order cubic order on the number of variables.
Thus, the primal is $O(M^3)$, and the dual is $O(N^3)$. Therefore **the dual is
advantageous for high-dimensional feature spaces**. 

Besides, the kernel trick allows the direct computation of kernels in the input space. Moreover, since in the solution $a_n \ne 0$ only for a subset of the $N$ training points (called **suppor vectors (SV)**, the prediction **does not require to evaluate the kernel versus all training set instances, but only against those in SV**, leading to:

\begin{equation*}
y(\pmb{x}) = \text{sign}(\sum_{x_n\text{ is SV}} a_nt_nk(\pmb{x},\pmb{x_n}) + b)
\end{equation*}

It is important to note that, **unlike linear regression with kernels, in this case the model must be trained to find which data are support vectors**. Then, **calculating the predictions requires evaluating the kernel for the point to be predicted against those support vectors** obtained during training.

Besides, SVMs also support **soft-margin** solutions, that is allowing
some points to lie in the margin, or even being in the wrong side of the separation boundary. For that points some form of penalty is included in the primal form, leading to slightly modified dual forms.

We are going how to use SVM with the implementation available in sklearn with the class **sklearn.svm.SVC** (SVC stands from Support Vector **Classification**, since other SVM variations are available for regression and even unsupervised learning). For large data sets (case of MNIST) the alternative **LinearSVC** class can be used.

In the following examples we will show how to apply this method for different data sets. Where possible, the instances which are support vectors are highlighted. Note also that the SVC implementation considers a regularization coefficient $1/C$ for $\|\pmb{w}\|^2$ in the optimization.

### Linearly separable data set

In [ ]:
np.random.seed(1)
r = 0.3*np.random.rand(50, 1) #0...1
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_0 = np.c_[1+r*3*np.cos(a), 5*r*np.sin(a)]

r = 0.3*np.random.rand(50, 1) #0.9...1.9
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_1 = np.c_[-1+r*2*np.cos(a), 5*r*np.sin(a)]

X = np.r_[X_0, X_1]
t = np.r_[np.zeros([50,1]),np.ones([50,1])].ravel()

# Plot data set
plt.figure(figsize=(6, 6))
plt.plot(X[t==0, 0], X[t==0, 1], "bs")
plt.plot(X[t==1, 0], X[t==1, 1], "g^")

plt.axis([-2, 2, -2, 2])
plt.show()

In [ ]:
def plot_svc_decision_boundary(svm_clf, xmin, xmax):
    w = svm_clf.coef_[0]
    b = svm_clf.intercept_[0]

    # At the decision boundary, w0*x0 + w1*x1 + b = 0
    # => x1 = -w0/w1 * x0 - b/w1
    x0 = np.linspace(xmin, xmax, 200)
    decision_boundary = -w[0]/w[1] * x0 - b/w[1]

    margin = 1/w[1]
    gutter_up = decision_boundary + margin
    gutter_down = decision_boundary - margin

    svs = svm_clf.support_vectors_
    plt.scatter(svs[:, 0], svs[:, 1], s=180, facecolors='#FFAAAA')
    plt.plot(x0, decision_boundary, "k-", linewidth=2)
    plt.plot(x0, gutter_up, "k--", linewidth=2)
    plt.plot(x0, gutter_down, "k--", linewidth=2)

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear',C=10) # The highest C, the more flexible the model
clf.fit(X, t)
print(f'The score (accurary) in the training set is {clf.score(X,t)*100:.02f}%')

# Plot the decision boundary
plt.figure(figsize=(8, 6))
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")
plot_svc_decision_boundary(clf, -2, 2)
plt.axis([-2, 2, -2, 2])
plt.show()

In red circles the SV are highlighted. Note that determining the class of a new point $\pmb{x}$ only requires kernel evaluation against these 6 points.

### Iris data set

In [ ]:
import numpy as np
import os
from sklearn import datasets
iris = datasets.load_iris()
# print(iris.DESCR) # Uncomment to show infor about the data set

# Let's create the feature and the target set
X = iris["data"][:, (2, 3)]  # Petal length and width
t = (iris["target"] == 2).astype(np.int)
t = t.reshape([X.shape[0],1])

# Since only two features are used, we can plot the data set
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

plt.figure(figsize=(8, 6))
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")

plt.text(3.5, 1.5, "Non Iris-Virginica", fontsize=14, color="b", ha="center")
plt.text(6.5, 2.3, "Iris-Virginica", fontsize=14, color="g", ha="center")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
plt.show()

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='linear')
clf.fit(X, t.ravel())
print(f'The score (accurary) in the training set is {clf.score(X,t)*100:.02f}%')

# Plot the decision boundary
plt.figure(figsize=(8, 6))
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")
plot_svc_decision_boundary(clf, 3, 7)
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
plt.show()

In this case, data is not linearly separable, and some points are allowed in the wrong side of the decision boundary. Note that more points are SVs in this case.

### Spherical vs. Torus data set

In [ ]:
np.random.seed(1)
r = np.random.rand(50, 1) #0...1
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_0 = np.c_[r*np.cos(a), r*np.sin(a)]

r = 0.9 + np.random.rand(50, 1) #0.9...1.9
a = 2*np.pi*np.random.rand(50,1) #0..2pi
X_1 = np.c_[r*np.cos(a), r*np.sin(a)]

X = np.r_[X_0, X_1]
t = np.r_[np.zeros([50,1]),np.ones([50,1])].ravel()

# Plot data set
plt.figure(figsize=(6, 6))
plt.plot(X[t==0, 0], X[t==0, 1], "bs")
plt.plot(X[t==1, 0], X[t==1, 1], "g^")

plt.axis([-2, 2, -2, 2])
plt.show()

In [ ]:
def plot_predictions(clf, axes):
    x0s = np.linspace(axes[0], axes[1], 100)
    x1s = np.linspace(axes[2], axes[3], 100)
    x0, x1 = np.meshgrid(x0s, x1s)
    X = np.c_[x0.ravel(), x1.ravel()]
    y_pred = clf.predict(X).reshape(x0.shape)
    y_decision = clf.decision_function(X).reshape(x0.shape)
    plt.contourf(x0, x1, y_pred, cmap=plt.cm.brg, alpha=0.2)
    plt.contourf(x0, x1, y_decision, cmap=plt.cm.brg, alpha=0.1)

In [ ]:
from sklearn.svm import SVC

clf = SVC(kernel='poly',degree=2)
clf.fit(X, t)
print(f'The score (accurary) in the training set is {clf.score(X,t)*100:.02f}%')

# Plot the decision boundary
plt.figure(figsize=(8, 6))
plt.plot(X[t.reshape(X.shape[0],)==0, 0], X[t.reshape(X.shape[0],)==0, 1], "bs")
plt.plot(X[t.reshape(X.shape[0],)==1, 0], X[t.reshape(X.shape[0],)==1, 1], "g^")
plot_predictions(clf, [-2, 2, -2, 2])
plt.axis([-2, 2, -2, 2])
plt.show()

### MNIST Data Set 

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
# print(mnist.DESCR) # Uncomment to show description
X, t = mnist["data"], mnist["target"] # N = 70000, D = 784 
X_train, X_test, t_train, t_test = X[:60000], X[60000:], t[:60000], t[60000:]
X_train = X_train/255
X_test = X_test/255

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")
    
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    images = [instance.reshape(size,size) for instance in instances]
    n_rows = (len(instances) - 1) // images_per_row + 1
    row_images = []
    n_empty = n_rows * images_per_row - len(instances)
    images.append(np.zeros((size, size * n_empty)))
    for row in range(n_rows):
        rimages = images[row * images_per_row : (row + 1) * images_per_row]
        row_images.append(np.concatenate(rimages, axis=1))
    image = np.concatenate(row_images, axis=0)
    plt.imshow(image, cmap = matplotlib.cm.plasma, **options)
    plt.axis("off")

plt.figure(figsize=(9,9))
example_images = X[:100]
plot_digits(example_images, images_per_row=10)
plt.show()

<b style="color:red"> This fitting requires a high-end computer (about 20 minutes in a machine with 12 cores and 64G of RAM). To execute and save classifier to disk uncomment the following lines. A precomputed model is available in AV. </b>

In [ ]:
#from sklearn.svm import LinearSVC
#clf = LinearSVC(max_iter=1000000) # Multiclass with one-vs-rest strategy
#clf.fit(X_train, t_train)
#clf = SVC(kernel='rbf')
#clf.fit(X_train, t_train)

#from joblib import dump
#dump(clf, 'SVM-MNIST-RBF.joblib')  # SAVE TO DISK

In [ ]:
# LOAD PRECOMPUTED SVM CLASSIFIER FROM DISK
# The file SVM-MNIST.joblib is available in AV
# Download and save in the same folder as the notebook

from joblib import load
clf = load('SVM-MNIST-RBF.joblib') 
print(clf)

<b style="color:red">Next cell requires approximately 10 minutes to calculate</b>

In [ ]:
print(f'The score (accurary) in the training set is {clf.score(X_train,t_train)*100:.02f}%')
print(f'The score (accuracy) in the test set is {clf.score(X_test,t_test)*100:.02f}%\n')

plot_digits(X_test[:10], images_per_row=10)
print('Labels predicted for the 10 first pictures in the test set')
print(clf.predict(X_test[:10]))
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

y_test = clf.predict(X_test)
conf_mx = confusion_matrix(t_test, y_test)
print(conf_mx)